In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [ ]:
! pip install  -r requirements.txt -q

In [ ]:
from transformers import *
from summarizer import Summarizer

# Load model, model config and tokenizer via Transformers
custom_config = AutoConfig.from_pretrained('monologg/biobert_v1.1_pubmed')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('monologg/biobert_v1.1_pubmed')
custom_model = AutoModel.from_pretrained("monologg/biobert_v1.1_pubmed", config=custom_config)


summed = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)


In [9]:
import os

docs = os.listdir("/root/Filtered")

for doc in docs:
    
    with open('/root/Filtered/'+ doc, "r") as in_f:
        body = in_f.read()
    
    summary = ''.join(summed(body, num_sentences = 5))
    
    path = "/root/Summary/" + doc[:-4] + "_SUMMARY.txt"
    
    with open(path, "w") as out_f:
        out_f.write(summary)

In [17]:
s3 = boto3.resource('s3')

bucket = sess.default_bucket()   

txtfiles = os.listdir("/root/Summary")

for txtfile in txtfiles:
    fpath = '/root/Summary/' + txtfile
    s3.meta.client.upload_file(fpath, bucket, txtfile)


In [11]:
bucket

'sagemaker-us-west-2-360604624615'